In [1]:
#-*- coding: windows-1250 -*-
import os
import glob
import pandas as pd

extension = 'csv'
os.chdir(r'c:\GIS\projekty\git_mirror\IMGW\dane_all')
result = glob.glob('*.{}'.format(extension))

lista_df =[]
for i in result:
    if "k_m_d" in str(i):
        df = pd.read_csv(i, header = None, encoding= 'windows-1250')
        lista_df.append(df)
        
df_polaczone = pd.concat(lista_df)
df_polaczone = df_polaczone.rename(columns={
    0: "kod_stacji",1: "nazwa_stacji",2: "year",3: "month", 4: "t_max",5: "status_tmax",
    6: "sr_t_max",  7: "status_sr_tmax",8: "t_min",9: "status_tmin",10: "sr_t_min",
    11: "status_sr_min", 12: "sr_temp_mies",
   13: "stat_stm",14: "min_temp_grunt",15: "stat_tmng",16: "opady_mies_srenia", 
    17: "status_opady_mies_srenia",
   18: "opady_max_doba",19: "st_opady_max_doba",20: "pierw_dzien_opad_maks", 
    21: "stat_opmx",
   22: "pierw_dzien_opad_mak",23: "ostat_dzien_opad_maks",24: "maks_pokrywy",
    25: "stat_pokrywy",
  26: "liczba_dni_pokrywa",27: "liczba_dni_deszcz", 28: "liczba_dni_snieg"})

# exrtacting interesting fields & date

In [2]:
#-*- coding: windows-1250 -*-
df_polaczone = df_polaczone[['nazwa_stacji', 'kod_stacji', 'year', 'month', 'sr_temp_mies']]

df_polaczone['data'] = df_polaczone['year'].astype(str) + "-" +df_polaczone['month'].astype(str) + "-01"
df_polaczone['data'] = pd.DatetimeIndex(df_polaczone['data'])
df_polaczone.set_index('data', inplace=True)

lista_stacji = df_polaczone.kod_stacji.unique()
lista_months = df_polaczone.month.unique()

# extracting month mean temp, for each station, for average reference period of 1961–1990

In [23]:
# select the refference period
df_polaczone_avg = df_polaczone[(df_polaczone['year'] >= 1961)]
df_polaczone_avg = df_polaczone_avg[(df_polaczone_avg['year'] <= 1990)]

lista_df_avg =[]
for month in lista_months:
    df_polaczone_mies_avg = df_polaczone_avg.loc[df_polaczone_avg['month'] == month]  # df for month within ref period    
    for stacja in lista_stacji:
        df_stacja_avg_ref = df_polaczone_mies_avg.loc[df_polaczone_mies_avg['kod_stacji'] == stacja] 
        df_stacja_avg_ref['avg_60_90'] = df_stacja_avg_ref.sr_temp_mies.mean() # mean temp for station, for month, ref period
        df_stacja_avg_ref['key'] = df_stacja_avg_ref['kod_stacji'].astype(str) + df_stacja_avg_ref['month'].astype(str) # key for join
        df_stacja_avg_ref= df_stacja_avg_ref[['key', 'avg_60_90']] 
        lista_df_avg.append(df_stacja_avg_ref)
    
df_avg_stacje = pd.concat(lista_df_avg)
df_avg_stacje = df_avg_stacje.drop_duplicates(subset=['key'])

C:\Users\marci\Anaconda2\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\marci\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


# join main table with the mean temp. from the avarage refference period, for each station

In [61]:
df_polaczone['key'] = df_polaczone['kod_stacji'].astype(str) + df_polaczone['month'].astype(str) # key for join
df_polaczone_avg = pd.merge(df_polaczone, df_avg_stacje, on='key')
df_polaczone_avg['anomalia'] = df_polaczone_avg['sr_temp_mies'] - df_polaczone_avg['avg_60_90']  # anomaly for each month-station

# Exporting GDF for each month for each year to seperate shapefile

In [60]:
from shapely.geometry import Point
import geopandas as gpd

lista_lat = range(1956,2020)
latacz = gpd.read_file(r'c:\Users\marci\git\iMGW\shp\latacz.shp') # patch shapefile  
df_xy = pd.read_csv(r'c:\GIS\projekty\git_mirror\IMGW\csv\stacje_located.csv') # import x,y locations for stations
                                                           
for month in lista_months:
    # create folder
    folder = os.path.join(r'c:\Users\marci\git\iMGW\output', str(month))
    if not os.path.exists(folder):
        os.makedirs(folder)
    df_mies = df_polaczone_avg.loc[df_polaczone_avg['month'] == month]
    for rok in lista_lat:    
        rok_format = str(int(rok))
        df_rok_mies = df_mies.loc[df_mies['year'] == rok]  
    
        # importig coordinates for stations
        result = pd.merge(df_rok_mies,
                         df_xy[['kod_stacji', 'latitude', 'longitude']],
                         on='kod_stacji')

        gdf = gpd.GeoDataFrame(
            result, geometry=gpd.points_from_xy(x=result.longitude, y=result.latitude),
            crs={'init':'epsg:4326'})

        # reprojection to PUWG1992
        gdf = gdf.to_crs({'init': 'epsg:2180'})
        
        liczba = gdf.shape[0]
        srednia = gdf.anomalia.mean()
        latacz['anomalia'] = srednia
        
        gdf_rok_mies_final = pd.concat([gdf, latacz], sort=True)
        filename_shp = os.path.join(folder,  str(rok) + ".shp") 
        gdf_rok_mies_final.to_file(driver = 'ESRI Shapefile', filename = filename_shp)